# 1. Create and run sample game

In [ ]:
from kaggle_environments import make, utils
from kaggle_environments.envs.hungry_geese.hungry_geese import renderer

env = make("hungry_geese", debug=True)
bot_list = ['greedy', 'random', 'greedy', 'random']

# 2. Save game states and veiw replay

In [ ]:
states = env.run(bot_list)
game_len = len(states)
env.render(mode="ipython", width=500, height=400)

# 3. Look at few last steps by built-in renderer()

In [ ]:
def view_result(state, bot_list):    
    print('bot      \tstatus \treward')
    print('----------------------------')
    for i, bot in enumerate(bot_list):
        print('{}:  \t{} \t{}'.format(bot if type(bot) is str else bot.__name__, 
                                     state[i]['status'],
                                     state[i]['reward']))
    print('----------------------------')

print(renderer(states[game_len-2], env))
view_result(states[game_len-2], bot_list)

print(renderer(states[game_len-1], env))
view_result(states[game_len-1], bot_list)

# 4. Create little viewer to simplify analysis

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle, Circle, Polygon
import json, requests

DIR = {'NORTH': (-1, 0),
       'SOUTH': (+1, 0),
       'EAST': (0, +1),
       'WEST': (0, -1)
      }
OPPOSITE = {'NORTH': 'SOUTH',
            'SOUTH': 'NORTH',
            'EAST': 'WEST',
            'WEST': 'EAST'
           }

class Viewer:
    SCALE = 1
    MARGIN = 0.05 * SCALE
    
    GEESE_COLORS = {0:{'body':'darkgray',
                       'head':'white',
                       'arrows':(0.75,0.75,0.75)
                      },
                    1:{'body':'blue',
                       'head':'cyan',
                       'arrows':(0, 0, 0.55)
                      },
                    2:{'body':'green',
                       'head':'lime',
                       'arrows':(0, 0.75, 0)
                      },
                    3:{'body':'red',
                       'head':'pink',
                       'arrows':(0.55, 0, 0)
                      }
                   }
    
    FOOD_COLOR = 'yellow'
    BACKGROUND_COLOR = (0.4,0.4,0.15)
    
    RIGHT_ARROW = [[0, 0], [-.2, -.2], [-.2, -.05], [-.4, -.05], [-.4, .05], [-.2, .05], [-.2, .2]]    
    LEFT_ARROW = [[-x, y] for [x, y] in RIGHT_ARROW]
    DOWN_ARROW = [[y, x] for [x, y] in RIGHT_ARROW]
    UP_ARROW = [[y, -x] for [x, y] in RIGHT_ARROW]    
    
    ARROWS = {'NORTH': UP_ARROW,
              'SOUTH': DOWN_ARROW,
              'EAST': RIGHT_ARROW,
              'WEST': LEFT_ARROW
             }
    
    def __init__(self, rows=7, columns=11):
        self.figure = plt.figure()
        
        for c in Viewer.GEESE_COLORS:
            plt.plot(-10,-10,Viewer.GEESE_COLORS[c]['body'], marker='o', markersize=10)
                        
        self.board_view = self.figure.add_subplot(111)
        self.board_view.set_facecolor('black')
        
        self.cell_width = Viewer.SCALE - Viewer.MARGIN*2
        self.cell_height = Viewer.SCALE - Viewer.MARGIN*2
        self.head_radius = Viewer.SCALE * 0.4
        self.food_radius = Viewer.SCALE * 0.2
        
        self.rows = rows
        self.columns = columns
        
        self.states = []
        self.legend = []
        self._draw_grid()
        
    def _norm(self, row_col, delta=(0,0)):
        row, col = row_col
        dr, dc = delta        
        r = (row + dr + self.rows) % self.rows
        c = (col + dc + self.columns) % self.columns        
        return r, c
    
    def _rect(self, row, col, color):
        x, y = col*Viewer.SCALE-Viewer.SCALE/2 + Viewer.MARGIN, row*Viewer.SCALE-Viewer.SCALE/2 + Viewer.MARGIN
        rect = Rectangle(xy=(x, y), width=self.cell_width, height=self.cell_height , color=color)
        #print (rect.__dict__)
        self.board_view.add_patch(rect)
    
    def _circle(self, row, col, color, radius):
        x, y = col*Viewer.SCALE, row*Viewer.SCALE
        c = Circle(xy=(x, y), radius=radius, color=color)
        self.board_view.add_patch(c)
        
    def _arrow(self, row, col, direction, color):        
        p = Polygon([[x+col, y+row] for [x,y] in Viewer.ARROWS[direction]], color=color)        
        self.board_view.add_patch(p)
    
    def show(self):        
        plt.axis([-0.6,10.6,6.6,-0.6])        
        plt.show()
        
    def _draw_grid(self):
        for r in range(self.rows):
            for c in range(self.columns):
                self._rect(r, c, Viewer.BACKGROUND_COLOR)
        
                
    def draw_geese(self, geese):
        for idx, goose in enumerate(geese):
            for j, (row, col) in enumerate(goose):
                if j == 0:
                    self._circle(row, col, Viewer.GEESE_COLORS[idx]['head'], self.head_radius)
                else:
                    self._rect(row, col, Viewer.GEESE_COLORS[idx]['body'])
                
    def draw_food(self, food):
        for row, col in food:
            self._circle(row, col, Viewer.FOOD_COLOR, self.food_radius)
            
    def set_states(self, states):
        self.states = states
        
    def load_from_url(self, url):
        json_file = requests.get(url)
        game_replay = json.loads(json_file.text)
 
        self.states = game_replay['steps']
        self.legend = game_replay['info']['TeamNames']                
        plt.legend(self.legend, framealpha=0.35, shadow=True, fontsize='large')    
        
    def draw_step(self, step):
        if self.states is not None and step in range(len(self.states)):
            plt.title('Step {}'.format(step + 1))
            
            prev_state = None
            current_state = self.states[step]
            next_state = None
            
            if step > 0:
                prev_state = self.states[step - 1]
            if step + 1 < len(self.states):
                next_state = self.states[step + 1]
             
            # geese            
            geese_pos = current_state[0]['observation']['geese']            
            geese_row_col = []
            for idx, goose in enumerate(geese_pos):
                geese_row_col.append([])
                for position in goose:
                    row, col = position // self.columns, position % self.columns                    
                    geese_row_col[idx].append((row, col))
            
            self.draw_geese(geese_row_col)
            
            # food
            food_pos = current_state[0]['observation']['food']
            food_row_col = []
            for position in food_pos:
                row, col = position // self.columns, position % self.columns
                food_row_col.append((row, col))
                
            self.draw_food(food_row_col)
            
            # action arrows
            # FROM direction:
            for idx, goose in enumerate(geese_row_col):
                if len(goose) > 0:
                    action = current_state[idx]['action']
                    if action is not None:
                        row, col = self._norm(goose[0], DIR[OPPOSITE[action]])
                        self._arrow(row, col, action, Viewer.GEESE_COLORS[idx]['arrows'])
            
            # TO direction:
            if next_state is not None:
                for idx, goose in enumerate(geese_row_col):
                    if len(goose) > 0:
                        action = next_state[idx]['action']
                        if action is not None:
                            row, col = self._norm(goose[0], DIR[action])
                            self._arrow(row, col, action, Viewer.GEESE_COLORS[idx]['arrows'])
        

# 5. Look at last steps one more time

In [ ]:
for s in range(game_len - 2, game_len):    
    v = Viewer()
    v.set_states(states)
    v.draw_step(s)
    v.show()

# 6. Load replay file from URL

In [ ]:
steps = 5
for s in range(195, 200):    
    v = Viewer()
    v.load_from_url('https://www.kaggleusercontent.com/episodes/13922785.json')    
    v.draw_step(s)
    v.show()
    